## Density and Velocity (method D)


In [ ]:
d = parse(jpsreport_inifile)
method_D =  d.getElementsByTagName('method_D')[0].attributes.items()[0][1]
cfiles = glob.glob(classical_fd_files)
ifiles = glob.glob(IFD_files)
ids_IFD = set([])
ids_FD = set([])
for f in ifiles:
    ids_IFD.add(int(f.split("id_")[-1].split(".")[0]))

sorted(ids_IFD)
for f in cfiles:
    ids_FD.add(int(f.split("id_")[-1].split(".")[0]))    

sorted(ids_FD)
# fps
o = open(cfiles[0])
header = o.readline()
o.close()
fps = float(header.split(":")[-1]) 

In [ ]:
def plot_density_velocity(Id):
    """
    method D
    file contain three columns: 
    >> frame density velocity <<
    """
    files =  glob.glob(os.path.join(
                                    jpsreport_ini_dir, 
                                    output_dir, 
                                    "Fundamental_Diagram", 
                                    "Classical_Voronoi", 
                                    "*id_{}.dat".format(Id)))
    fig, axs = plt.subplots(2, 1)
    for f in files:
        data = np.loadtxt(f)
        # v(t)
        axs[0].plot(data[:, 0]/fps, data[:, 1], label="{}".format(Id))
        axs[0].set(xlabel=r'$t\; [s]$', ylabel='$\\rho\; [1/m{-2}$]')
        axs[0].legend()
        # rho(t)
        axs[1].plot(data[:, 0]/fps, data[:, 2], label="{}".format(Id))
        axs[1].set(xlabel=r'$t\; [s]$', ylabel='$v\; [m/s$]')
        axs[1].legend()

    plt.tight_layout()

In [ ]:
def plot_IFD_density_velocity(Id):
    """
    method D
    file contain many columns: 
    Frame PersID x, y, z, density, velocity, Voronoi Polygon, Intersection Polygon
    """
    fig, axs = plt.subplots(2, 1)
    files = glob.glob(os.path.join(
                                    jpsreport_ini_dir, 
                                    output_dir, 
                                    "Fundamental_Diagram", 
                                    "IndividualFD", 
                                    "*id_{}.dat".format(Id)))
    for f in files:
        df = read_IFD(f)
        # v(t)
        axs[0].plot(df['Frame']/fps, df['rho'], ".", label="{}".format(Id))
        axs[0].set(xlabel=r'$t\; [s]$', ylabel='$\\rho\; [1/m{-2}$]')
        axs[0].legend()
        # rho(t)
        axs[1].plot(df['Frame']/fps, df['vel'], ".", label="{}".format(Id))
        axs[1].set(xlabel=r'$t\; [s]$', ylabel='$v\; [m/s$]')
        axs[1].legend()

    plt.tight_layout()

In [ ]:
def plot_profiles(Id):
    density_files = os.path.join(field_dir, "density",  "Prf_*id_{}_*".format(Id))
    velocity_files = os.path.join(field_dir, "velocity",  "Prf_*id_{}_*".format(Id))
    v_Voronoi = glob.glob(velocity_files)
    f_Voronoi = glob.glob(density_files)
    # get the shape of the matrices
    shape = np.shape(np.loadtxt(f_Voronoi[0]))
    density = np.zeros(shape)
    velocity = np.zeros(shape)
    geo_filename= os.path.join(os.path.dirname(jpsreport_inifile), 
                               d.getElementsByTagName('geometry')[0].attributes.items()[0][1])
    xml_datei = open(geo_filename, "r")
    geo_xml = parse(xml_datei)
    xml_datei.close()
    geominX, geomaxX, geominY, geomaxY = geo_limits(geo_xml)

    geometry_wall = read_subroom_walls(geo_xml)
    geometry_obst = read_obstacle(geo_xml)
#  -------- density
    for density_file in f_Voronoi:
        if os.path.exists(density_file):    
            density += np.loadtxt(density_file)

    density = density / len(f_Voronoi)
#  --------- velocity
    for velocity_file in v_Voronoi:
        if os.path.exists(velocity_file):    
            velocity += np.loadtxt(velocity_file)

    velocity = velocity / len(f_Voronoi)
    flow = density * velocity
   
 # plot
    fig1, ax1 = plt.subplots()
    ax1.set_aspect('equal')
    
    fig2, ax2 = plt.subplots()
    ax2.set_aspect('equal')
    
    fig3, ax3 = plt.subplots()
    ax3.set_aspect('equal')
    
    for g in geometry_obst.keys():
        ax1.add_patch(ppolygon(geometry_obst[g], color='black'))
        ax2.add_patch(ppolygon(geometry_obst[g], color='black'))
        ax3.add_patch(ppolygon(geometry_obst[g], color='black'))

    for gw in geometry_wall.keys():
        ax1.plot(geometry_wall[gw][:, 0], geometry_wall[gw][:, 1], color='black', lw=4)
        ax2.plot(geometry_wall[gw][:, 0], geometry_wall[gw][:, 1], color='black', lw=4)
        ax3.plot(geometry_wall[gw][:, 0], geometry_wall[gw][:, 1], color='black', lw=4)

    im1 = ax1.imshow(density, 
                    cmap=cm.jet,
                    interpolation='nearest', origin='lower',
                    vmin=0, vmax=np.amax(density),
                    extent=[geominX, geomaxX, geominY, geomaxY]) 
    
    im2 = ax2.imshow(velocity, 
                    cmap=cm.jet,
                    interpolation='nearest', origin='lower',
                    vmin=0, vmax=np.amax(velocity),
                    extent=[geominX, geomaxX, geominY, geomaxY])
    
    im3 = ax3.imshow(flow,
                    cmap=cm.jet, interpolation='nearest', origin='lower',
                    vmin=0, vmax=np.amax(flow),
                    extent=[geominX, geomaxX, geominY, geomaxY])
    
    ax1.set_xlabel("x [m]")
    ax1.set_ylabel("y [m]")
    divider1 = make_axes_locatable(ax1)
    cax1 = divider1.append_axes("right", size="3.5%", pad=0.3)
    cb1 = plt.colorbar(im1, cax=cax1)
    cb1.set_label('Density [$m^{-2}$]')
    
    ax2.set_xlabel("x [m]")
    ax2.set_ylabel("y [m]")
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes("right", size="3.5%", pad=0.3)
    cb2 = plt.colorbar(im2, cax=cax2)
    cb2.set_label('Velocity [$m/s$]')
    
    ax3.set_xlabel("x [m]")
    ax3.set_ylabel("y [m]")
    divider3 = make_axes_locatable(ax3)
    cax3 = divider3.append_axes("right", size="3.5%", pad=0.3)
    cb3 = plt.colorbar(im3, cax=cax3)
    cb3.set_label('Flow [$1/m \cdot s$]')

In [ ]:
def plot_method_D(choice):
    if choice == 'FD':
        widgets.interact(plot_density_velocity, Id=widgets.Dropdown(options=ids_FD, description='M-Area', disabled=False))
    elif choice == 'IFD':
        widgets.interact(plot_IFD_density_velocity, Id=widgets.Dropdown(options=ids_IFD, description='M-Area', disabled=False))
    elif choice == 'Profiles':
        files = glob.glob(os.path.join(field_dir, "density",  "Prf_*.dat"))
        ids = set([])
        for f in files:
            ids.add(int(f.split("id_")[-1].split("_")[0]))
        sorted(ids)    
        widgets.interact(plot_profiles, Id=widgets.Dropdown(options=ids, description='M-Area', disabled=False))

    elif choice == 'Voronoi polygons':
        for f in ifiles:
            df = read_IFD(f)
            Id = int(f.split("id_")[-1].split(".")[0])
            #IFD_plot_polygon_rho(df)
            # todo
            
    else:
        print(">> all")
        # todo?

In [ ]:
if method_D.lower() == "true":
    choice = widgets.Dropdown(
       options=['FD', 'IFD', 'Profiles', 'Voronoi polygons', 'all'],
        value='FD',
        description='Plot: ',
        disabled=False,
    )
    widgets.interact(plot_method_D, choice=choice, description='Choice', disabled=False);
else:
    print("Method D not active. Nothing to plot..")    